In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import os 
from scipy.signal import butter, sosfiltfilt, resample
os.chdir('/home/kkotzen/research/PPG_sleepstaging_orion2 /')
from pathlib import Path
from src.parsing.MESAParser import MESAParser

dl = MESAParser()
patients = dl.database_patient_IDs[0:10]

this_patient = patients[0]
ah = dl.load_apnea_hypopnea(this_patient)
sleep = dl.load_sleep(this_patient)
ppg_input = dl.load_signal(this_patient, 'Pleth')

In [2]:
dl = MESAParser()
patients = dl.database_patient_IDs[0:10]

t_sleep = np.arange(0, len(sleep)*30, 30)
t_ah = np.arange(0, len(ah))

fig, ax = plt.subplots(2,1, sharex=True)
ax[0].plot(t_sleep, sleep)
ax[1].plot(t_ah, ah)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
fs = 256
resample_fs = 16
F = fs/resample_fs
highcut = 2
order=8
sos = butter(order, [highcut / (fs / 2)], 'lowpass', output='sos')

ppg = sosfiltfilt(sos, ppg_input, axis=0)
ppg = resample(ppg, int(len(ppg)/F))
fs = resample_fs

In [ ]:
from scipy.fft import fft, dct
from scipy import signal 

def _time_series_subsequences(ts, window, hop=1):
    assert len(ts.shape) == 1
    shape = (int(int(ts.size - window) / hop + 1), window)
    strides = ts.strides[0] * hop, ts.strides[0]
    return np.lib.stride_tricks.as_strided(ts, shape=shape, strides=strides)

win, hop = 30, 30
f, t, Sxx = signal.spectrogram(ppg, fs, nperseg = fs*win, noverlap=0, window='tukey')
Sxx[Sxx > np.quantile(Sxx, 0.99)] = np.quantile(Sxx, 0.99)
for i in range(Sxx.shape[1]):
    Sxx[:,i] = (Sxx[:,i]-np.mean(Sxx[:,i]))/np.std(Sxx[:,i])

print(np.max(Sxx[200]))
plt.close('all')
plt.pcolormesh(t, f, (Sxx)*100)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
    
# X = _time_series_subsequences(ppg, 32*30, 32*30)
# X_dct = np.zeros_like(X)
# for i, x in enumerate(X):
#     win, hop = 30, 30
    

In [ ]:
ppg = ppg[0:39000*fs]
t_sleep = np.arange(0, len(sleep)*30, 30)
t_ah = np.arange(0, len(ah))
t_ppg =  np.arange(0, len(ppg)/fs, 1/fs)

fig, ax = plt.subplots(4,1, sharex=True)
ax[0].plot(t_sleep, sleep)
ax[1].plot(t_ah, ah)
ax[2].plot(t_ppg, ppg)
# ax[3].pcolormesh(t, f[0:40], (Sxx[0:40,:]))

widths = np.arange(1, 31)
cwtmatr = signal.cwt(ppg, signal.ricker, widths)
ax[3].plot(t_ppg, cwtmatr[5,:], label='5')
ax[3].plot(t_ppg, cwtmatr[15,:], label='15')
ax[3].plot(t_ppg, cwtmatr[20,:], label='20')
ax[3].legend()
# ax[3].imshow(cwtmatr, cmap='PRGn', aspect='auto',
#            vmax=abs(cwtmatr).max(), vmin=-abs(cwtmatr).max(), origin='lower')

plt.show()